# Playground for Transformers!

### Attention is all you need 
(https://arxiv.org/abs/1706.03762)

### For software vulnerability detection GYM

This is a minimal example of this **CRAZY** idea!

### Dataset

Import

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import re
import torch.backends.cudnn as cudnn
import torchtext
import matplotlib.pyplot as plt
import time
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import SGD,Adam
import torch.nn.functional as F
import random
from gensim.models.word2vec import Word2Vec
import torchtext.vocab as vocab

Load playset dataset

In [2]:
dataset = pd.read_pickle('playset(0.25.2).pickle')

View dataset

In [3]:
dataset

,functionSource,combine
93792,"go_file_opener_open (GOFileOpener const *fo, g...",False
79770,updatePathMap(bool left_level) {\n\tPoint from...,False
66999,interpret_tilde(const char* path) {\n stati...,False
44284,"checkVarExp(\n Absyn *node,\n Ta...",True
49515,will_have_skip_worktree(const struct cache_ent...,True
...,...,...
96701,"AVLTree_insert(AVLTree * tree, void * data)\n{...",False
67815,"remove_hook(const char *name, hookfn fn)\n{\n\...",False
88363,"output_def(dico_stream_t str, struct gcide_db ...",False
65929,getState(\n\t\tFLMUINT\t\tuiFieldID)\n\t{\n\t\...,False


In [4]:
dataset.describe(include='all')

,functionSource,combine
count,100000,100000
unique,100000,2
top,literal_value (cb_tree x)\n{\n\tif (x == cb_sp...,True
freq,1,50000


In [5]:
dataset.functionSource[1]

'checkCapture2() const\n{\n    for(int i=6;i<48;i++)\n    {\n        switch(board[i])\n        {\n        case MAN2:\n            if(board[i+5]==MAN1 || board[i+5]==KING1)\n                if(board[i+10]==FREE) return true;\n            if(board[i+6]==MAN1 || board[i+6]==KING1)\n                if(board[i+12]==FREE) return true;\n            break;\n        case KING2:\n            if(board[i-6]==MAN1 || board[i-6]==KING1)\n                if(board[i-12]==FREE) return true;\n            if(board[i-5]==MAN1 || board[i-5]==KING1)\n                if(board[i-10]==FREE) return true;\n            if(board[i+5]==MAN1 || board[i+5]==KING1)\n                if(board[i+10]==FREE) return true;\n            if(board[i+6]==MAN1 || board[i+6]==KING1)\n                if(board[i+12]==FREE) return true;\n        }\n    }\n    return false;\n}'

### Prepare data (in JSON)

#### Clean & Preprocessing

In [6]:
def clean(code):
    ## Remove newlines & tabs
    code = re.sub('(\n)|(\\\\n)|(\\\\)|(\\t)|(/)','',code)
    ## Remove code comments
    code = re.sub(r'/\*(.|[\r\n])*?\*/','',code)
    ## Mix split (characters and words)
    splitter = ' +|(;)|(\()|(==)|(\))|(=)|(\+)|(\-)|(\[)|(\])|(<)|(>)|({)|(#)|(\")'
    code = re.split(splitter,code)
    ## Remove None type
    code = list(filter(None, code))
    code = list(filter(str.strip, code))
    ## Return list of tokens
    return(code)

In [7]:
## Clean the codes
dataset.functionSource = dataset.functionSource.apply(clean)

In [8]:
## Change labels boolean to 1 and 0
dataset.iloc[:,1] = np.multiply(dataset.iloc[:,1],1)

In [9]:
## Change column name
dataset = dataset.rename(columns={'functionSource':'codes', 'combine':'label'})

In [10]:
false = dataset[dataset.iloc[:,1]==0]
true = dataset[dataset.iloc[:,1]==1]

In [11]:
## Split to train,test,valid
train = false[0:4000].append(true[0:4000])
test  = false[4000:4500].append(true[4000:4500])
valid = false[4500:5000].append(true[4500:5000])

## Shuffle
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)

In [12]:
## Save to json
train.to_json('.data/train_min.json', orient='records',lines=True)
test.to_json('.data/test_min.json', orient='records',lines=True)
valid.to_json('.data/valid_min.json', orient='records',lines=True)

In [13]:
## Define the field

CODES = torchtext.data.Field(batch_first=True)
LABEL = torchtext.data.LabelField(dtype=torch.int64)
fields = {'codes': ('codes', CODES), 'label': ('label', LABEL)}

In [14]:
## Import the min data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_min.json',
                                        validation = 'valid_min.json',
                                        test = 'test_min.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the ALL data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_all.json',
                                        validation = 'valid_all.json',
                                        test = 'test_all.json',
                                        format = 'json',
                                        fields = fields)

In [15]:
## Testing
print(vars(valid_data[0]))

{'codes': ['__ecereConstructor___ecereNameSpace__eda__Report', '(', 'struct', '__ecereNameSpace__ecere__com__Instance', '*', 'this', ')', '{', 'struct', '__ecereNameSpace__eda__Report', '*', '__ecerePointer___ecereNameSpace__eda__Report', '=', '(', 'struct', '__ecereNameSpace__eda__Report', '*', ')', '(', 'this', '?', '(', '(', '(', 'char', '*', ')', 'this', ')', '+', '__ecereClass___ecereNameSpace__eda__Report', '-', '>', 'offset', ')', ':', '0', ')', ';', '__ecerePointer___ecereNameSpace__eda__Report', '-', '>', 'groupings', '=', '__ecereNameSpace__ecere__com__eInstance_New', '(', '__ecereClass___ecereNameSpace__ecere__com__Array_TPL_eda__Grouping_', ')', ';', '__ecereNameSpace__ecere__com__eInstance_IncRef', '(', '__ecerePointer___ecereNameSpace__eda__Report', '-', '>', 'groupings', ')', ';', '{', '__ecereProp___ecereNameSpace__eda__Report_Set_pageFormat', '(', 'this,', '1', ')', ';', '}return', '0x1', ';', '}'], 'label': 1}


Doneeeeeeeeeeeeeeeee !

### Vocabulary-related preparation

In [16]:
##### Build the vocabulary

MAX_VOCAB_SIZE = 10000

CODES.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [17]:
print(f"Unique tokens in TEXT vocabulary: {len(CODES.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 2


In [18]:
## Most common word
print(CODES.vocab.freqs.most_common(20))

[('(', 137341), (')', 137335), (';', 126428), ('-', 63469), ('=', 62781), ('>', 49012), ('"', 36587), ('{', 30774), ('*', 25960), ('if', 25784), ('}', 19525), ('+', 18237), ('0', 17330), ('[', 16280), (']', 16266), ('return', 13171), (',', 12923), ('i', 10581), ('==', 10369), ('1', 10295)]


In [19]:
print(CODES.vocab.itos[:10])
print(LABEL.vocab.stoi)

['<unk>', '<pad>', '(', ')', ';', '-', '=', '>', '"', '{']
defaultdict(None, {0: 0, 1: 1})


In [21]:
## place into iterators
train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = 64,
    sort = False)

### Prepare Word2Vec (Optional)

In [10]:
corpus = pd.read_json('.data/train.json',orient='records',lines=True)

w2v = Word2Vec(corpus.codes, size=300, workers=16, sg=1, min_count=3)
w2v.save('.data/node_w2v_128')

In [10]:
w2v = Word2Vec.load('.data/node_w2v_128')
w2v.wv.vectors

array([[ 0.04268723, -0.01990928, -0.10372822, ...,  0.34928635,
        -0.24622028, -0.02363101],
       [ 0.06655177, -0.08706249, -0.11346684, ...,  0.2967248 ,
        -0.16500187, -0.10260527],
       [ 0.10059763, -0.0993171 , -0.14234892, ...,  0.3913037 ,
        -0.22237949,  0.02339004],
       ...,
       [-0.0031671 ,  0.01939397, -0.00094254, ..., -0.06062187,
        -0.0873417 ,  0.10190531],
       [-0.03719744,  0.02801778,  0.02174594, ..., -0.05577604,
        -0.07265704, -0.00079473],
       [ 0.01092949, -0.03061507, -0.045645  , ..., -0.06999503,
        -0.15679213,  0.11291362]], dtype=float32)

### Init all the non important stuffs

In [22]:
seed = 1234
torch.manual_seed(seed)
#device = torch.device("cpu");
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
cudnn.benchmark = True
cudnn.enabled = True

### Transformer class (with LSTM)

In [23]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer,self).__init__()
        
        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=104)
        self.encode_layer = nn.TransformerEncoderLayer(d_model=104,
                                                       nhead=8,
                                                       dim_feedforward=1024,
                                                       dropout=0.1,
                                                       activation='relu')
        self.trans_encoder = nn.TransformerEncoder(self.encode_layer,
                                                   num_layers=2)
        self.lstm1 = nn.LSTM(input_size=104,
                            hidden_size=104,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=False)
        self.fc1 = nn.Linear(104,2)
    
    def forward(self,x):
        x = self.embed(x)
        x = self.trans_encoder(x)
        r_out, (h_n, h_c) = self.lstm1(x)
        x = r_out[:,-1,:]
        x = self.fc1(x)
        #x = F.log_softmax(x,dim=1)
        return(x)
    
model = Transformer()
model.to(device)
print(model)

Transformer(
  (embed): Embedding(10002, 104)
  (encode_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=104, out_features=104, bias=True)
    )
    (linear1): Linear(in_features=104, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=1024, out_features=104, bias=True)
    (norm1): LayerNorm((104,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((104,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (trans_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=104, out_features=104, bias=True)
        )
        (linear1): Linear(in_features=104, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_featur

### LSTM class

In [14]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        
        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=50)
        self.lstm1 = nn.LSTM(input_size=50,
                            hidden_size=64,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=False)
        #self.fc1 = nn.Linear(128,64)
        self.fc2 = nn.Linear(64,2)
        
    
    def forward(self,x):
        x = self.embed(x)
        r_out, (h_n, h_c) = self.lstm1(x,None)
        x = r_out[:,-1,:]
        #x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.log_softmax(x,dim=1)
        return(x)
    
model = LSTM()
model.to(device)
print(model)

LSTM(
  (embed): Embedding(10002, 50)
  (lstm1): LSTM(50, 64, batch_first=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)


### Load Word2Vec weights to embeddings

In [14]:
weights = torch.FloatTensor(w2v.wv.vectors)
weights = weights.to(device)
model.embed = model.embed.from_pretrained(weights)
#model.embed = model.embed.weight.data.copy_(weights)

### Parameters

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,989,786 trainable parameters


### Prepare accuracy function

In [25]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [26]:
def softmax_accuracy(probs,all_labels):
    def getClass(x):
        return(x.index(max(x)))
    
    all_labels = all_labels.tolist()
    probs = pd.Series(probs.tolist())
    all_predicted = probs.apply(getClass)
    all_predicted.reset_index(drop=True, inplace=True)
    vc = pd.value_counts(all_predicted == all_labels)
    try:
        acc = vc[1]/len(all_labels)
    except:
        if(vc.index[0]==False):
            acc = 0
        else:
            acc = 1
    return(acc)

### Loss functions

In [27]:
## Define optimizer
optimizer = SGD(model.parameters(), lr = 5.0)
#optimizer = Adam(model.parameters(), lr = 0.001, eps=1)

## Define loss function
#criterion = nn.BCELoss().to(device) ## Sigmoid activation function
#criterion = nn.NLLLoss().to(device) ### Log_softmax activation
criterion = nn.CrossEntropyLoss().to(device) ## No activation function bcs softmax included

### Training

In [ ]:
epochs=50

for e in range(epochs):
    running_acc = 0
    running_loss = 0
    timer = time.time()

    for batch in train_iterator:
        batch.codes = batch.codes.to(device)
        batch.label = batch.label.to(device)
        optimizer.zero_grad()
        output = model(batch.codes)
        loss = criterion(output, batch.label)
        loss.backward()
        optimizer.step()
        acc = softmax_accuracy(output,batch.label)
        running_acc += acc.item()
        running_loss += loss.item()
        print('Done Batch')
    else:
        with torch.no_grad():
            running_acc_val = 0
            running_loss_val = 0
            for batch in valid_iterator:
                batch.codes = batch.codes.to(device)
                batch.label = batch.label.to(device)
                output_val = model(batch.codes)
                loss_val = criterion(output_val,batch.label)
                acc_val = softmax_accuracy(output_val,batch.label)
                running_acc_val += acc_val.item()
                running_loss_val += loss_val.item()
        
        print("Epoch {} - Training acc: {:.6f} -Training loss: {:.6f} - Val acc: {:.6f} - Val loss: {:.6f} - Time: {:.4f}s".format(e+1, running_acc/len(train_iterator), running_loss/len(train_iterator), running_acc_val/len(valid_iterator), running_loss_val/len(valid_iterator), (time.time()-timer)))

### Testing

In [ ]:
### SOFTMAX

import sklearn.metrics
from sklearn.metrics import confusion_matrix

with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        batch.codes = batch.codes.to(device)
        batch.label = batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)

In [ ]:
#### BINARY

import sklearn.metrics
from sklearn.metrics import confusion_matrix

with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        batch.codes = batch.codes.to(device)
        batch.label = batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = binary_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += torch.round(output_test).tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_pred)
print('Confusion matrix: \n',confusion)

# RESULTS